In [2]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive

# Path to the uploaded TensorFlow Lite model
tflite_model_path = '/content/deeplabv3.tflite'  # Path to the uploaded model file

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

# Allocate tensor memory for the interpreter
interpreter.allocate_tensors()

# Function to process input image and apply TensorFlow Lite DeepLabV3 model
def segment_image(image_path):
    # Load image
    image = cv2.imread(image_path)

    # Check if the image is loaded correctly
    if image is None:
        raise ValueError("Image not found or unable to load image.")

    # Resize image to fit DeepLabV3 model (257x257 for this version)
    input_image = cv2.resize(image, (257, 257))  # Correct input size for DeepLabV3

    # Normalize the image (from 0-255 to 0-1 range)
    input_image = input_image.astype(np.float32)
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension
    input_image = np.divide(input_image, 255.0)  # Normalize to 0-1 range

    # Get input and output details from the TensorFlow Lite model
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Set the input tensor for the interpreter
    interpreter.set_tensor(input_details[0]['index'], input_image)

    # Run the inference
    interpreter.invoke()

    # Get the output tensor
    output = interpreter.get_tensor(output_details[0]['index'])

    # Convert the output to a mask
    output_mask = output.squeeze()  # Remove unnecessary dimensions
    output_mask = np.argmax(output_mask, axis=-1)  # Get the class with the highest score (background/foreground)

    return image, output_mask

# Function to apply Gaussian Blur to the background
def apply_bokeh_effect(image, segmentation_mask, blur_strength=15):
    # Ensure blur_strength is an odd number greater than 1
    if blur_strength % 2 == 0:
        blur_strength += 1  # If it's even, make it odd

    # Resize segmentation mask to match the original image size
    segmentation_mask_resized = cv2.resize(segmentation_mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Create a mask for the subject (foreground)
    subject_mask = segmentation_mask_resized > 0.5  # Threshold mask to create a binary mask

    # Create a copy of the original image for the subject
    subject = np.copy(image)
    subject[~subject_mask] = 0  # Zero out the background pixels

    # Create a copy of the original image for the background
    background = np.copy(image)

    # Apply Gaussian Blur to the background (background is where subject_mask is 0)
    background_blurred = cv2.GaussianBlur(background, (blur_strength, blur_strength), 0)

    # Combine the subject and blurred background using the mask
    final_image = np.where(subject_mask[:, :, None], subject, background_blurred)

    return final_image

# Load image (upload the image to your Colab environment first)
image_path = '/content/K_D.jpg'  # Image path updated (make sure the image is uploaded to /content/)
image, segmentation_mask = segment_image(image_path)

# Function to update the image based on slider input
def update_bokeh_strength(blur_strength):
    # Apply bokeh effect (adjust the blur strength for the background)
    final_image = apply_bokeh_effect(image, segmentation_mask, blur_strength)

    # Show the original image and final image side by side
    plt.figure(figsize=(12, 6))

    # Plot the original image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis('off')

    # Plot the final image with bokeh effect
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))
    plt.title("Image with Bokeh Effect")
    plt.axis('off')

    # Display the images
    plt.show()

# Create an interactive slider for blur strength
blur_slider = widgets.IntSlider(value=15, min=1, max=31, step=2, description='Blur Strength:')
interactive_plot = interactive(update_bokeh_strength, blur_strength=blur_slider)

# Display the slider
display(interactive_plot)


interactive(children=(IntSlider(value=15, description='Blur Strength:', max=31, min=1, step=2), Output()), _do…